# Deploy Llama2 7B Chat with LMI 
## TensorRT-LLM

### Quantization: None
See serving.properties

In [4]:
model_name = "llama2-7b-chat"
model_filename = "trtllm-" + model_name + ".tar.gz"
s3_prefix = "trtllm-" + model_name
instance_type = "ml.g5.12xlarge" #"ml.g4dn.12xlarge"

In [ ]:
!pip install -U sagemaker

In [ ]:
%pip install transformers

In [28]:
import time
import sagemaker
from sagemaker.model import Model
from sagemaker import image_uris
from sagemaker import serializers, deserializers

role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
bucket = sess.default_bucket()
region = sess._region_name

In [ ]:
# upload model code archive to S3
!rm {model_filename} 2> /dev/null
!tar -czf {model_filename} {s3_prefix}
s3_artifact = sess.upload_data(model_filename, bucket, s3_prefix)
s3_artifact

In [30]:
def create_model(_model_name, model_s3_url):
    # Get the LMI image uri
    image_uri = image_uris.retrieve(
        framework="djl-tensorrtllm",
        region=region,
        version="0.25.0"
    )
    hub = {
        'HUGGING_FACE_HUB_TOKEN': 'hf_XXXXXXXXXXXXXXXXXXXXXXX'
    }
    model = Model(
        image_uri=image_uri,
        model_data=model_s3_url,
        role=role,
        name=_model_name,
        sagemaker_session=sess,
        env=hub
    )
    return model

In [31]:
def deploy_model(model, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=_endpoint_name,
        container_startup_health_check_timeout=900
        #endpoint_logging=True
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sess,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [32]:
endpoint_name = model_name + "-" + time.strftime("%Y%m%d-%H%M%S")
endpoint_name

'llama2-7b-chat-20240101-184601'

In [33]:
model = create_model(endpoint_name, s3_artifact)

In [34]:
print(model.image_uri)

763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.25.0-tensorrtllm0.5.0-cu122


In [37]:
predictor = deploy_model(model, endpoint_name)

Using already existing model: llama2-7b-chat-20240101-184601


-------------!

In [92]:
from timeit import default_timer as timer
from transformers import LlamaTokenizerFast
tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

In [97]:
def test_model(prompt):
    start = timer()
    res = predictor.predict(
        data={ 
            "inputs" : prompt,
            "parameters": {"max_new_tokens":400}
        }
        #custom_attributes = "accept_eula=true"
    )
    end = timer()
    print(res)
    print("Elapsed: ", end-start)
    print("Tokens: ", len(tokenizer.encode(res['generated_text'])))

In [98]:
test_model("What is SageMaker LMI?")

{'generated_text': "\n\nSageMaker LMI (Learning Model Insights) is a feature of Amazon SageMaker that provides a set of tools and algorithms to help machine learning (ML) practitioners interpret and understand the performance of their ML models. SageMaker LMI helps users to identify potential issues with their models, such as bias or drift, and to improve their model's performance over time.\n\nSageMaker LMI provides a range of capabilities, including:\n\n1. Model interpretability: SageMaker LMI provides tools to help users understand how their ML models are making predictions, including feature importance, partial dependence plots, and SHAP values.\n2. Model monitoring: SageMaker LMI allows users to monitor their ML models in real-time, detecting potential issues such as bias or drift, and providing recommendations for improvement.\n3. Model optimization: SageMaker LMI provides algorithms to help users optimize their ML models, such as hyperparameter tuning and model pruning.\n4. Expl

In [99]:
test_model("What are the recommended steps to train for an AWS Solutions Architect certification?")

{'generated_text': "\n\nAWS Solutions Architect certification is a professional certification offered by Amazon Web Services (AWS) that validates an individual's expertise in designing and deploying scalable, secure, and efficient cloud-based systems on AWS. To prepare for the certification exam, follow these steps:\n\n1. Understand the exam format: The AWS Solutions Architect certification exam consists of 60 multiple-choice questions, divided into two sections: Designing and Deploying Systems on AWS (30 questions) and Architecting on AWS (30 questions).\n2. Familiarize yourself with AWS services: Study the various AWS services, including Compute, Storage, Database, Security, Networking, and Analytics. Understand the features, pricing, and use cases for each service.\n3. Learn about AWS architectural design patterns: Study the AWS architectural design patterns, such as the N-tier architecture, the Microservices architecture, and the Serverless architecture. Understand how to design an

In [100]:
test_model("write a blog post explaining how to select a university for an MBA. Elaborate on the various aspects one should consider.")

{'generated_text': "\nIntroduction:\nChoosing the right university for an MBA program is a crucial decision that can have a significant impact on one's career and future. With so many universities offering MBA programs, it can be overwhelming to narrow down the options. In this blog post, we will discuss the various aspects one should consider when selecting a university for an MBA program.\n\n1. Program Reputation:\nThe reputation of the MBA program is one of the most important factors to consider. Look for universities that have a good reputation in the business world and are accredited by reputable accrediting agencies. Check the rankings of the university's MBA program in various publications, such as Forbes, Bloomberg, or The Economist.\n2. Curriculum:\nThe curriculum of the MBA program should align with your career goals and interests. Look for universities that offer a diverse range of courses, including finance, marketing, operations, and strategy. Also, check if the program of

### Cleanup Resources

In [13]:
predictor.delete_endpoint()

In [14]:
model.delete_model()